In [26]:
import os
from os.path import join
import img2pdf
import re
from tqdm import tqdm
from file_function import write_log_csv_upload

############################################
path = r"D:\extraKey\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\1.한울가람_헬프\! 매입전자료(한울가람스캔자료)\원인서류\법인"
path_out = r"D:\extraKey\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\1.한울가람_헬프\! 매입전자료(한울가람스캔자료)\원인서류\법인 합본"
p_extension = re.compile('(jpeg|jpg|bmp|gif|png|tif|tiff)$', re.I) 
############################################
path_success_log = r'D:\0.전산\1.진행중과업\파일서버개편\구스캔폴더 작업log\success'
path_fail_log = path

success = []
fail = []
notImg = []

dir_list = [d.name for d in os.scandir(path) if d.is_dir()]
total=len(dir_list)

for d in tqdm(dir_list, total=total) :
    temp = []
    file_list = os.listdir(join(path, d))
    for f in file_list :
        if p_extension.search(f) :
            temp.append(join(path, d, f))
        else :
            notImg.append([path, d, f])
            continue
    
    if len(temp)>0 :
        try :
            output = join(path_out, d + " 원인서류 포함 매각사 자료모음.pdf")
            sn = 0
            while os.path.exists(output) :
                sn += 1
                output = join(path_out, d + " 원인서류 포함 매각사 자료모음" + str(sn) + ".pdf")

            with open(output,"wb") as f:
                f.write(img2pdf.convert(temp))

            success.append([d, output])
        except Exception as e :
            fail.append([path, d, output, e.__class__])

loss = total - len(success) -len(fail)
print(loss, "개 누수")
print(f"전체 : {total}개, 파일변환 : {len(success)}개, 실패{len(fail)}개")
print(f"{len(notImg)}개 이미지 아닌 파일 있음")

write_log_csv_upload(success, path_success_log, "파일병합 성공")
write_log_csv_upload(fail, path_fail_log, "파일병합 성공")

100%|██████████| 312/312 [03:28<00:00,  1.50it/s]

0 개 누수
전체 : 312개, 파일변환 : 312개, 실패0개
0개 이미지 아닌 파일 있음
log 파일이 비어있습니다. path : D:\extraKey\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\1.한울가람_헬프\! 매입전자료(한울가람스캔자료)\원인서류\법인
